In [1]:
import tensorflow as tf
import keras
print("tf=",tf.__version__)
print("keras=",keras.__version__)

Using TensorFlow backend.


tf= 1.3.0
keras= 2.0.8


In [2]:
from keras.models import Model
from keras.models import load_model
from keras import backend as K

from tensorflow.python.tools import freeze_graph
import os

# All new operations will be in test mode from now on.
K.set_learning_phase(0)

model_file = "ResNet50-UdacityRealandSimMix-Best-val_acc-1.0.hdf5"
model = load_model(model_file)
print(model.summary())

print([node.op.name for node in model.outputs])

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
image_input (InputLayer)     (None, 224, 224, 3)       0         
_________________________________________________________________
resnet50 (Model)             multiple                  23587712  
_________________________________________________________________
global_average_pooling2d_1 ( (None, 2048)              0         
_________________________________________________________________
dropout_1 (Dropout)          (None, 2048)              0         
_________________________________________________________________
fc1 (Dense)                  (None, 1024)              2098176   
_________________________________________________________________
dropout_2 (Dropout)          (None, 1024)              0         
_________________________________________________________________
predictions (Dense)          (None, 4)                 4100      
Total para

In [4]:
# Serialize the model and get its weights, for quick re-building.
config = model.get_config()
weights = model.get_weights()

# Re-build a model where the learning phase is now hard-coded to 0.
new_model = Model.from_config(config)
new_model.set_weights(weights)

print([node.op.name for node in new_model.outputs])

['predictions_1/Softmax']


In [10]:
temp_dir = "graph"
checkpoint_prefix = os.path.join(temp_dir, "saved_checkpoint")
checkpoint_state_name = "checkpoint_state"
input_graph_name = "input_graph.pb"
output_graph_name = "ResNet50-UdacityRealandSimMix-Best-val_acc-1.pb"

# Temporary save graph to disk without weights included.
saver = tf.train.Saver()
checkpoint_path = saver.save(K.get_session(), checkpoint_prefix, global_step=0, latest_filename=checkpoint_state_name)
tf.train.write_graph(K.get_session().graph, temp_dir, input_graph_name)

input_graph_path = os.path.join(temp_dir, input_graph_name)
input_saver_def_path = ""
input_binary = False
#changed to new model
#output_node_names = "predictions/Softmax" # model dependent
output_node_names = "predictions_1/Softmax" # model dependent
restore_op_name = "save/restore_all"
filename_tensor_name = "save/Const:0"
output_graph_path = os.path.join(temp_dir, output_graph_name)
clear_devices = False

# Embed weights inside the graph and save to disk.
freeze_graph.freeze_graph(input_graph_path, input_saver_def_path,
                          input_binary, checkpoint_path,
                          output_node_names, restore_op_name,
                          filename_tensor_name, output_graph_path,
                          clear_devices, "")

INFO:tensorflow:Restoring parameters from graph\saved_checkpoint-0
INFO:tensorflow:Froze 322 variables.
Converted 322 variables to const ops.
1252 ops in the final graph.


In [11]:
def load_graph(frozen_graph_filename):
    # We load the protobuf file from the disk and parse it to retrieve the 
    # unserialized graph_def
    with tf.gfile.GFile(frozen_graph_filename, "rb") as f:
        graph_def = tf.GraphDef()
        graph_def.ParseFromString(f.read())

    # Then, we import the graph_def into a new Graph and returns it 
    with tf.Graph().as_default() as graph:
        # The name var will prefix every op/nodes in your graph
        # Since we load everything in a new graph, this is not needed
        tf.import_graph_def(graph_def, name="tl_classify")
    return graph

In [12]:
    graph = load_graph(output_graph_path)

    # We can verify that we can access the list of operations in the graph
    for op in graph.get_operations():
        print(op.name)
        # prefix/image_input
        # ...
        # prefix/predictions/Softmax

tl_classify/image_input_1
tl_classify/conv1_1/kernel
tl_classify/conv1_1/kernel/read
tl_classify/conv1_1/bias
tl_classify/conv1_1/bias/read
tl_classify/bn_conv1_1/gamma
tl_classify/bn_conv1_1/gamma/read
tl_classify/bn_conv1_1/beta
tl_classify/bn_conv1_1/beta/read
tl_classify/bn_conv1_1/moving_mean
tl_classify/bn_conv1_1/moving_mean/read
tl_classify/bn_conv1_1/moving_variance
tl_classify/bn_conv1_1/moving_variance/read
tl_classify/res2a_branch2a_1/kernel
tl_classify/res2a_branch2a_1/kernel/read
tl_classify/res2a_branch2a_1/bias
tl_classify/res2a_branch2a_1/bias/read
tl_classify/bn2a_branch2a_1/gamma
tl_classify/bn2a_branch2a_1/gamma/read
tl_classify/bn2a_branch2a_1/beta
tl_classify/bn2a_branch2a_1/beta/read
tl_classify/bn2a_branch2a_1/moving_mean
tl_classify/bn2a_branch2a_1/moving_mean/read
tl_classify/bn2a_branch2a_1/moving_variance
tl_classify/bn2a_branch2a_1/moving_variance/read
tl_classify/res2a_branch2b_1/kernel
tl_classify/res2a_branch2b_1/kernel/read
tl_classify/res2a_branch2b_1/

In [13]:
    import cv2
    import numpy as np
    from keras.applications import resnet50
    
    labels = np.array(['green', 'none', 'red', 'yellow'])
    
    # We access the input and output nodes 
    x = graph.get_tensor_by_name('tl_classify/image_input_1:0')
    y = graph.get_tensor_by_name('tl_classify/predictions_1/Softmax:0')
    #z = graph.get_tensor_by_name('tl_classify/bn_conv1_1/keras_learning_phase:0')
    
    image = cv2.imread('0_resized.png')
    image = cv2.resize(image, (224, 224))
    cv2.imshow('image', image)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
    np_image_data = np.asarray(image)
    np_final = np.expand_dims(np_image_data, axis=0)
    np_final = resnet50.preprocess_input(np_final.astype('float64'))
          
    # We launch a Session
    with tf.Session(graph=graph) as sess:
        #yhat = sess.run(y, feed_dict={x: np_final, z: 0})
        yhat = sess.run(y, feed_dict={x: np_final})
    
    yhat = yhat[0]
    y_class = yhat.argmax(axis=-1)
    
    print("Detected, {:d} = {} with probability = {:4.2f} %.".format(y_class, labels[y_class], yhat[y_class]*100))

Detected, 0 = green with probability = 98.01 %.


In [30]:
print(K.learning_phase())

0
